In [5]:
!pip install medcat==1.2.7


  Using cached spacy_langdetect-0.1.2-py3-none-any.whl (5.0 kB)
  Using cached langdetect-1.0.7.zip (998 kB)
  Created wheel for langdetect: filename=langdetect-1.0.7-py3-none-any.whl size=993461 sha256=fe0531e4abda4db4a9f65d16b691a1fed7ae1d2bde63986eb566fbb11ce55e4d
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\c1\6d\ab\bf9ecd1ab14dd236da586dfd0d4b008e2e803e571cf2229c26
Successfully built langdetect


In [ ]:
!pip install spacy_langdetect

In [7]:
!python -m spacy download en_core_web_md

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [11]:
from medcat.cat import CAT
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector
from collections import defaultdict
import re
import json

#@Language.factory("language_detector")
#def get_lang_detector(nlp, name):
   #return LanguageDetector()

# Download the model_pack from the models section in the github repo.
DATA_DIR = "C:/Users/HP/nlptools/data/"
model_pack_path = DATA_DIR + "medmen_wstatus_2021_oct.zip"
cat = CAT.load_model_pack(model_pack_path)
print("--------------------------------------")

keys = []
values = []
with open('data/input.txt', 'r') as file:
    text = file.read().replace('\n', '')
type_ids_filter = ['T033']
cui_filters = set()
for type_ids in type_ids_filter:
  cui_filters.update(cat.cdb.addl_info['type_id2cuis'][type_ids])
cat.cdb.config.linking['filters']['cuis'] = cui_filters
doc=cat(text)



Found an existing unziped model pack at: C:/Users/HP/nlptools/data\medmen_wstatus_2021_oct, the provided zip will not be touched.
{
  "Model ID": null,
  "Last Modifed On": null,
  "History (from least to most recent)": [],
  "Description": "No description",
  "Source Ontology": null,
  "Location": null,
  "MetaCAT models": {},
  "Basic CDB Stats": {},
  "Performance": {
    "ner": {},
    "meta": {}
  },
  "Important Parameters (Partial view, all available in cat.config)": {
    "config.ner['min_name_len']": {
      "value": 3,
      "description": "Minimum detection length (found terms/mentions shorter than this will not be detected)."
    },
    "config.ner['upper_case_limit_len']": {
      "value": 3,
      "description": "All detected terms shorter than this value have to be uppercase, otherwise they will be ignored."
    },
    "config.linking['similarity_threshold']": {
      "value": 0.2,
      "description": "If the confidence of the model is lower than this a detection will b

In [20]:
idx=0
for ent in doc.ents:
    if str(ent)!="detection":
        keys.append('Covid-Result')
        values.append(str(ent))
    print(ent)
doc2 = cat(text)

print("--------------------------------------")

nlp = spacy.load('en_core_web_md')

text1= nlp(text)
for w in text1.ents:
    if w.label_=='DATE':
        keys.append('DATE')
        s = re.sub(r'\|', '',  w.text)
        values.append(s)
        print(w.text)
        break
print("--------------------------------------")
text2 = nlp(text)
for w in text2.ents:
  if w.label_=='PERSON':
    keys.append('PERSON')
    values.append(w.text)
    print(w.text)
    break
print("--------------------------------------")


def patientID(text):
  ID_REG = re.compile(r'[a-zA-Z0-9\.\-+_]{14}')
  IDs = re.findall(ID_REG,text)
  patientID = ",".join(IDs)
  patientID = patientID.split(',')
  patientID = patientID[0]
  return patientID

patientID=patientID(text)
list_words= text.split()
index =list_words.index("ID")
pre_IDs=list_words[index-3:index+3]
for i in pre_IDs:
  if i==patientID:
    ID=i
keys.append('PATIENT ID')
values.append(patientID)
print(patientID)
print("--------------------------------------")
dictionary = dict(zip(keys, values))
json_object = json.dumps(dictionary, indent = 4)
jsonFile = open("data/output.json", "w")
jsonFile.write(json_object)
jsonFile.close()
print(json_object)
print("--------------------------------------")

#nlp.add_pipe('language_detector', last=True)
#doc = nlp(text)
#print(doc._.language)
#print("--------------------------------------")

Not Detected
Negative
--------------------------------------
| Oct 30, 2020
--------------------------------------
MMG20200057563
--------------------------------------
MMG20200057563
--------------------------------------
{
    "Covid-Result": "Negative",
    "DATE": " Oct 30, 2020",
    "PERSON": "MMG20200057563",
    "PATIENT ID": "MMG20200057563"
}
--------------------------------------


In [22]:
#Validating the extracted details
import json
import re
with open("data/output.json") as file:
    data_dict = json.load(file)
print(data_dict)


class main():
    def test_result(self):
        result=data_dict["Covid-Result"]
        if type(result)!=str:
            raise TypeError("result must be a string")
        self.assertEqual(result=="Positive" or "Negative", True)

    def test_date(self):
        date=data_dict["DATE"]
        if not re.match('^ (?:(((Jan(uary)? | Ma(r(ch)? | y) | Jul(y)? | Aug(ust)? | Oct(ober)? | Dec(ember)?)\ 31) | ((Jan(uary)? | Ma(r(ch)? | y) | Apr(il)? | Ju((ly?) | (ne?)) | Aug(ust)? | Oct(ober)? | (Sept | Nov | Dec)(ember)?)\ (0?[1-9] | ([12]\d) | 30)) | (Feb(ruary)?\ (0?[1-9] | 1\d | 2[0-8] | (29(?=, \ ((1[6-9] |[2-9]\d)(0[48] | [2468][048] | [13579][26]) | ((16 | [2468][048] |[3579][26])00)))))))\, \ ((1[6 - 9] | [2 - 9]\d)\d{2}))', date):
            return False
        return True

        #self.assertEqual(data_dict["DATE"] == , True)
    def test_person(self):
        username=data_dict["PERSON"]
        minlen=3
        if type(username) != str:
            raise TypeError("username must be a string")
        if minlen < 1:
            raise ValueError("minlen must be at least 1")

        # Usernames can't be shorter than minlen
        if len(username) < minlen:
            return False
        # Usernames can only use letters, numbers, dots and underscores
        if not re.match('^[a-z0-9._]*$', username):
            return False
        # Usernames can't begin with a number
        if username[0].isnumeric():
            return False
        return True

    def test_sampleID(self):
        sampleID=data_dict["PATIENT ID"]
        if not re.match('^[a-zA-Z0-9\.\-+_]{14}',sampleID):
            return False
        return True

if __name__ == '__main__':
    main()
    print("Extracted Details are validated!")


{'Covid-Result': 'Negative', 'DATE': ' Oct 30, 2020', 'PERSON': 'MMG20200057563', 'PATIENT ID': 'MMG20200057563'}
Extracted Details are validated!
